# Importar Blibliotecas

In [0]:
import cvxpy as cvx
import copy
from heapq import *
import numpy as np
import itertools
counter = itertools.count() 

# Criação Da Classe

In [0]:
class BBTreeNode():
    def __init__(self, vars = set(), constraints = [], objective=0, bool_vars=set()):
        self.vars = vars
        self.constraints = constraints
        self.objective = objective
        self.bool_vars = bool_vars
        self.children = []
    def buildProblem(self):
        prob = cvx.Problem(cvx.Minimize(self.objective), self.constraints) #i put Minimize, just so you know that I'm assuming it
        return prob
    def is_integral(self):
        print (all([abs(v.value - 1) <= 1e-3 or abs(v.value - 0) <= 1e-3 for v in self.bool_vars]))
        return all([abs(v.value - 1) <= 1e-3 or abs(v.value - 0) <= 1e-3 for v in self.bool_vars])



    def is_fractional(self):
      self.fracionario = []
      for num in self.bool_vars:
        if not str(round(num.value, 2)).replace('.','').isdigit(): return 
        if(float(round(num.value, 2)) != int(round(num.value, 2))):
          self.fracionario.append(num)
      #print(fracionario)
      if (len(self.fracionario) == 0):
        return False
      return self.fracionario

    

    def branch(self):
        children = []
        for b in [0,1]:
                n1 = copy.deepcopy(self) #yeesh. Not good performance wise, but is simple implementation-wise
                v = n1.heuristic() #dangerous what if they don't do the same one? I need to do it here though because I need access to copied v.
                n1.constraints.append( v == b ) # add in the new binary constraint
                n1.children = []
                n1.bool_vars.remove(v) #remove binary constraint from bool var set
                n1.vars.add(v) #and add it into var set for later inspection of answer
                #self.children.append(n1)   # eventually I might want to keep around the entire search tree. I messed this up though
                children.append(n1)             
        return children


    def heuristic(self):
        mais_proximo = 1e30
        #fracionarios = self.is_fractional()
        print(self.fracionario)
        for v in self.fracionario:
          print(round((v.value * 100 ),4))
          if (round(v.value, 4) > 0.5):
            x = range(round((int(v.value) * 100 ), 4) ,50 , -5)
          elif (round(v.value, 4) < 0.5):
            x = range (round((int(v.value) * 100 ), 4), 50, 5)
          elif (round(v.value, 4) == 0.5):
            return v
          if (len(x) <= mais_proximo):
            mais_proximo = v.value
            var_mais_proxima = v
        return var_mais_proxima

          
            
      


    def bbsolve(self):
        root = self
        res = root.buildProblem().solve()
        heap = [(res, next(counter), root)]
        bestres = 1e20 # a big arbitrary initial best objective value
        bestnode = root # initialize bestnode to the root
        print(heap)
        nodecount = 0
        while len(heap) > 0: 
            nodecount += 1 # for statistics
            print("Heap Size: ", len(heap))
            _, _, node = heappop(heap)
            prob = node.buildProblem()
            res = prob.solve()
            print("Result: ", res)
            if prob.status not in ["infeasible", "unbounded"]:
                if res > bestres - 1e-3: #even the relaxed problem sucks. forget about this branch then
                    print("Relaxed Problem Stinks. Killing this branch.")
                    pass
                elif not node.is_fractional(): #if a valid solution then this is the new best
                        print("New Best Integral solution.")
                        bestres = res
                        bestnode = node
                else: #otherwise, we're unsure if this branch holds promise. Maybe it can't actually achieve this lower bound. So branch into it
                    new_nodes = node.branch()
                    for new_node in new_nodes:
                        heappush(heap, (res, next(counter), new_node ) )  # using counter to avoid possible comparisons between nodes. It tie breaks
        print("Nodes searched: ", nodecount)      
        return bestres, bestnode

# Manipulando

In [0]:
matriz = []
arquivo = open('problema.txt','r')
for linha in arquivo:
  valores = linha.split()
  matriz.append(valores) #ler do arquivo as linhas e coloca na matriz, para manipulações futuras
arquivo.close()



for i in range(len(matriz)):
  for j in range (len(matriz[i])):
    matriz[i][j] = float(matriz[i][j]) #converte as string lida no arquivo para inteiro
matriz[0][0] = int(matriz[0][0])
matriz[0][1] = int(matriz[0][1])
print(matriz)


[[4, 2], [1.5, 2.0, 3.0, 0.8], [7.0, 8.0, 30.0, 6.0, 3.0], [550.0, 300.0, 400.0, 250.0, 50.0]]


In [0]:

tratamento = matriz.pop(0)
print (tratamento)
#separando variavel e restrição
variaveis = int(tratamento[0])
restricao = int(tratamento[1])
#colocando a lista da função objetivo na variavel

funcao_objetivo = matriz.pop(0)
matriz_restricao = matriz



[4, 2]


In [0]:
x = cvx.Variable(variaveis)
constraints = []
for i in range (restricao):
  constraints += [matriz_restricao[i][:-1]*x >= matriz_restricao[i][len(matriz_restricao[i])-1]] # cria as restrições
constraints +=  [x >= 0, x <=1 ] # relaxação entre 0 e 1, nos reais.
objective = funcao_objetivo * x
bool_vars = {x[i] for i in range(variaveis)} 


In [13]:
root = BBTreeNode(constraints = constraints, objective= objective, bool_vars = bool_vars)
res, sol = root.bbsolve()
print(sorted(list([(v.name(), v.value) for v in sol.bool_vars] + [(v.name(), v.value) for v in sol.vars] ) ))

[(0.3117647059446477, 23, <__main__.BBTreeNode object at 0x7ff390ccf518>)]
Heap Size:  1
Result:  0.3117647059446477
[Expression(AFFINE, UNKNOWN, ()), Expression(AFFINE, UNKNOWN, ())]
8.8235
5.8824
[Expression(AFFINE, UNKNOWN, ()), Expression(AFFINE, UNKNOWN, ())]
8.8235
5.8824
Heap Size:  2
Result:  0.40000000003044
[Expression(AFFINE, UNKNOWN, ())]
50.0
[Expression(AFFINE, UNKNOWN, ())]
50.0
Heap Size:  3
Result:  3.000000000065813
New Best Integral solution.
Heap Size:  2
Result:  0.6428571433976683
[Expression(AFFINE, UNKNOWN, ())]
42.8571
[Expression(AFFINE, UNKNOWN, ())]
42.8571
Heap Size:  3
Result:  0.8000000009900686
New Best Integral solution.
Heap Size:  2
Result:  0.7500000001150365
[Expression(AFFINE, UNKNOWN, ())]
37.5
[Expression(AFFINE, UNKNOWN, ())]
37.5
Heap Size:  3
Result:  1.5000000003998069
Relaxed Problem Stinks. Killing this branch.
Heap Size:  2
Result:  inf
Heap Size:  1
Result:  2.0000000000010756
Relaxed Problem Stinks. Killing this branch.
Nodes searched:  